In [1]:
# part of the code is from the Tutorial 6, and for preprocessing part, since I write with teammate Yiyun Ding, so part of the code is similar.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
from torchsummary import summary
import sys
import tarfile
import cv2
from IPython.display import display, Image
from __future__ import print_function
import numpy as np
from six.moves import range
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
# PyTorch
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler, TensorDataset
import torch.nn as nn
from timeit import default_timer as timer
from PIL import UnidentifiedImageError, Image
import pandas as pd

In [3]:
# extract tar.gz fromat file from google drive
def extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  tar = tarfile.open('/content/drive/My Drive/'+ filename)
  sys.stdout.flush()
  tar.extractall()
  tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  print(data_folders)
  return data_folders

data_filename = 'notMNIST_small.tar.gz'
data_folders = extract(data_filename) 

['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


In [4]:
mainfolders = ['test', 'train', 'valid']
subfolders = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
path = "/content/notMNIST_small/"
# create folders and subfolders for training, validation and testing
for i in mainfolders:
  try:
      os.mkdir(path + i)
  except:
      print(path + i)
for i in mainfolders:
  for j in subfolders:
    try: 
      os.mkdir(path + i + '/' + j)
    except:
      print( path + i + '/' + j)
# divide images into the three created folders and each folder has ten subfolders
for subfolder in subfolders:
  image_files = os.listdir('notMNIST_small/' + subfolder)
  i = 0
  # index 0 -1499 images into training set
  for i in range(1500):
      shutil.copyfile('notMNIST_small/{}/{}'.format(subfolder, image_files[i]), 
                      'notMNIST_small/train/{}/{}'.format(subfolder, image_files[i]))
  # index 1500 -1599 images into validation set
  for i in range(1500, 1600):
    shutil.copyfile('notMNIST_small/{}/{}'.format(subfolder, image_files[i]), 
                      'notMNIST_small/valid/{}/{}'.format(subfolder, image_files[i]))
  # put rest images into test set
  for i in range(1600, len(image_files)):
    shutil.copyfile('notMNIST_small/{}/{}'.format(subfolder, image_files[i]), 
                      'notMNIST_small/test/{}/{}'.format(subfolder, image_files[i]))
# for images cannot load, remove the images
for route in mainfolders:
  for subfolder in subfolders:
    subfolder_path = 'notMNIST_small/' + route + '/' + subfolder
    image_files = os.listdir(subfolder_path)
    for image in image_files:
      image_path = subfolder_path + '/' + image
      try:
        im = Image.open(image_path)
        im.load()
      except UnidentifiedImageError:
        os.remove(image_path)
        print(f'remove {image_path}')

remove notMNIST_small/test/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png
remove notMNIST_small/train/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png


In [5]:
training_dir = "/content/notMNIST_small/train"
validation_dir = "/content/notMNIST_small/test"
test_dir = "/content/notMNIST_small/valid"
batch_size = 128
data = {
    'train':
        datasets.ImageFolder(
            root=training_dir, 
            transform=transforms.Compose(
                [transforms.ToTensor(), 
                transforms.Lambda(lambda x: x[0, :, :].reshape(784))
                ]
            )
        ),
    'val':
        datasets.ImageFolder(
            root=validation_dir, 
            transform=transforms.Compose(
                [transforms.ToTensor(), 
                transforms.Lambda(lambda x: x[0, :, :].reshape(784))
                ]
            )
        ),
    'test':
        datasets.ImageFolder(
            root=test_dir, 
            transform=transforms.Compose(
                [transforms.ToTensor(), 
                transforms.Lambda(lambda x: x[0, :, :].reshape(784))
                ]
            )
        ),
}

dataloaders = {
    'train': DataLoader(data['train'], batch_size = batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size = batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size = batch_size, shuffle=True)
}

In [6]:
input_size = 784
hidden_sizes = 1000
output_size = 10
#build model
model = nn.Sequential(nn.Linear(input_size, hidden_sizes),
                      # uncomment for dropout
                      # nn.Dropout(0.5),
                      nn.ReLU(),
                      # uncomment next two line for second layer and divide hidden size by 2
                      # nn.Linear(hidden_sizes, hidden_sizes),
                      # nn.ReLU(),
                      nn.Linear(hidden_sizes, output_size),
                      nn.Softmax(dim=1))
print(model)



Sequential(
  (0): Linear(in_features=784, out_features=1000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1000, out_features=10, bias=True)
  (3): Softmax(dim=1)
)


In [7]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0002)
summary(
        model, input_size=(3, 784), batch_size=128, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [128, 3, 1000]         785,000
              ReLU-2             [128, 3, 1000]               0
            Linear-3               [128, 3, 10]          10,010
           Softmax-4               [128, 3, 10]               0
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.15
Forward/backward pass size (MB): 5.92
Params size (MB): 3.03
Estimated Total Size (MB): 10.10
----------------------------------------------------------------


In [8]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          test_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        test_loader (PyTorch dataloader): test dataloader for testing accuracy
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0
        test_loss = 0.0

        train_acc = 0
        valid_acc = 0
        test_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)
            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Test loop
                for data, target in test_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Test loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    test_loss += loss.item() * data.size(0)

                    # Calculate test accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    test_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)
                test_loss = test_loss / len(test_loader.dataset)
                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)
                test_acc = test_acc / len(test_loader.dataset)

                history.append([train_loss, valid_loss, test_loss, train_acc, valid_acc, test_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f}% \t Validation Loss: {valid_loss:.4f}% \t Test Loss: {test_loss:.4f}%'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}% \t Validation Accuracy: {100 * valid_acc:.2f}% \t Test Accuracy: {100 * test_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'test_loss', 'train_acc',
                                'valid_acc', 'test_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'test_loss', 'train_acc', 'valid_acc', 'test_acc'])
    return model, history
    

In [9]:
save_file_name = 'onelayer-finetuned.pt'
train_on_gpu = False
model, history = train(
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    dataloaders['test'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=100,
    print_every=2)

Starting Training from Scratch.



RuntimeError: ignored

In [ ]:
plt.figure(figsize=(8, 6))
for c in ['train_acc','valid_acc','test_acc']:
    plt.plot(
        100 * (1-history[c]), label= f'{c[:-4]}_error')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.title('Training, Validation and Test error')